In [21]:
import numpy as np
import pandas as pd
import numpy as np
import os
import sys
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio
import wave
from scipy.io.wavfile import write
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
#import tensorflow as tf
#from tf.keras.models import Sequential
#from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
#from keras.utils import np_utils, to_categorical
#from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout

In [2]:
from sklearn.neural_network import MLPClassifier

In [50]:
Features= pd.read_csv("../speech_emotion_reco/data/features.csv")

In [51]:
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.022190,0.574264,0.483570,0.460849,0.481321,0.555484,0.593383,0.527021,0.487356,0.539324,...,0.000017,5.487894e-06,1.681743e-06,1.302462e-06,1.284306e-06,1.304934e-06,1.355323e-06,1.416036e-06,1.307451e-06,sad
1,0.063119,0.676197,0.628908,0.634546,0.656816,0.704702,0.666063,0.597292,0.570871,0.612779,...,0.021895,2.096577e-02,2.190924e-02,2.180943e-02,2.149669e-02,2.173487e-02,2.182789e-02,2.222465e-02,2.047032e-02,sad
2,0.025224,0.492119,0.549708,0.484783,0.479518,0.517123,0.570607,0.615393,0.522012,0.452447,...,0.000026,7.272163e-06,3.031210e-06,1.611539e-06,1.207497e-06,1.187471e-06,1.176641e-06,1.092101e-06,9.572191e-07,sad
3,0.018410,0.635455,0.514304,0.476947,0.516035,0.538225,0.572309,0.577087,0.545820,0.521525,...,0.000002,4.997832e-07,2.244980e-07,2.392561e-07,2.810812e-07,3.590882e-07,5.512045e-07,8.656837e-07,1.010199e-06,sad
4,0.018681,0.652667,0.541908,0.507164,0.547772,0.562931,0.586111,0.578713,0.550591,0.530536,...,0.000114,1.217496e-04,1.168074e-04,1.163628e-04,1.118637e-04,1.123569e-04,1.165087e-04,1.152994e-04,1.155533e-04,sad


In [55]:
Features=Features[Features.labels != 'unknown']
Features=Features[Features.labels != 'surprise']

In [57]:
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

In [58]:
Features['labels'].value_counts()

happy      5772
disgust    5769
fear       5769
sad        5769
angry      5769
neutral    5685
Name: labels, dtype: int64

In [59]:
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [60]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(pd.DataFrame(X), pd.DataFrame(Y), random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((25899, 162), (25899, 6), (8634, 162), (8634, 6))

In [10]:
## PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=90)
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=90, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [11]:
print(sum(pca.explained_variance_ratio_))

0.9660104353282594


In [12]:
x_train_pca= pca.transform(x_train)

In [13]:
x_test_pca= pca.transform(x_test)


In [61]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((25899, 162), (25899, 6), (8634, 162), (8634, 6))

In [62]:
#MLP clasiifier sans pca
#52 % moins over fittint model= MLPClassifier(hidden_layer_sizes=(200,), max_iter=300)
# 55% model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (500,), learning_rate = 'adaptive', max_iter = 600)
#model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (400,), learning_rate = 'adaptive', max_iter = 500)
# 54% model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (500,), learning_rate = 'adaptive', max_iter = 900)
#53% model = MLPClassifier(alpha = 0.09, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (500,), learning_rate = 'adaptive', max_iter = 600)
#56% model = MLPClassifier(alpha = 0.03, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (700,), learning_rate = 'adaptive', max_iter = 800,  validation_fraction=0.2)
#54model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (700,), learning_rate = 'adaptive', max_iter = 800,  validation_fraction=0.2)
# 52% et 70% model= MLPClassifier(hidden_layer_sizes=(400,), max_iter=600, alpha = 0.1, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)
#53% 69% avec standardisation model= MLPClassifier(hidden_layer_sizes=(500,), max_iter=600, alpha = 0.1, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive', validation_fraction=0.2)
#44% 54% model= MLPClassifier(hidden_layer_sizes=(500,), max_iter=600, alpha = 0.2, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)

#70% 52 % 
model= MLPClassifier(hidden_layer_sizes=(500,), max_iter=600, alpha = 0.1, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive')


In [63]:
hist= model.fit(x_train, y_train)

In [64]:
## train accuracy
y_pred_train = model.predict(x_train)
accuracy_train = accuracy_score(y_true=y_train, y_pred=y_pred_train)
accuracy_train

0.6896405266612611

In [65]:
#test accuracy
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
accuracy

0.5186472087097521

In [23]:
## classifier PCA
# 40% 50% model= MLPClassifier(hidden_layer_sizes=(500,), max_iter=600, alpha = 0.01, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)
#42% 52% model= MLPClassifier(hidden_layer_sizes=(800,), max_iter=1000, alpha = 0.01, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)
#39% 47% model= MLPClassifier(hidden_layer_sizes=(2000,), max_iter=3000, alpha = 0.01, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)
#42% 52% model= MLPClassifier(hidden_layer_sizes=(1000,), max_iter=1200, alpha = 0.01, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)
#75% 51% model= MLPClassifier(hidden_layer_sizes=(300,), max_iter=500, alpha = 0.01, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)
#74% 52% model= MLPClassifier(hidden_layer_sizes=(400,), max_iter=500, alpha = 0.05, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)
model= MLPClassifier(alpha = 0.04, batch_size = 256, epsilon = 1e-08,learning_rate = 'adaptive',validation_fraction=0.2)



In [15]:
### grid search
from sklearn.model_selection import GridSearchCV
param= {"hidden_layer_sizes":[500,700,800],"max_iter":[500,600,800], "alpha":[0.01,0.3,0.06]}

gr=GridSearchCV(model,param,return_train_score=True, scoring="accuracy", n_jobs=-1)

In [16]:
hist_grid=gr.fit(x_train_pca, y_train)

In [19]:
hist_grid.cv_results_


{'mean_fit_time': array([1679.53095846, 1606.5600884 , 1913.75112658, 2219.1079155 ,
        2180.67874541, 2157.52992768, 2650.0130394 , 2707.94848433,
        2498.85229883,  697.39323826,  715.29972115,  760.37582855,
         690.64130692,  856.08863974,  897.92352443,  958.98217869,
         870.59202857,  647.23898907,  709.46445966,  584.9426856 ,
         705.4737587 ,  820.96376743,  759.89354258,  859.92752724,
         979.14883513,  985.70525923,  486.33232622]),
 'std_fit_time': array([229.22533515, 284.23655527, 478.28405227, 225.607089  ,
        198.81760663, 175.30708081, 231.39534841, 364.44015187,
        219.02107392, 133.74109883,  57.85616153,  77.21472241,
        103.67019187, 198.33243247, 118.26864731, 137.73625879,
        210.99859687, 101.30177736,  76.70494928,  81.06860197,
         52.8601292 ,  70.69354606, 165.13373036, 178.67181626,
        134.20214841, 170.24735568,  81.96983738]),
 'mean_score_time': array([0.27806811, 0.29505854, 0.2865447 , 0.395

In [20]:
print(hist_grid.best_params_)

{'alpha': 0.01, 'hidden_layer_sizes': 800, 'max_iter': 500}


In [84]:
hist= model.fit(x_train_pca, y_train)

In [85]:
## train accuracy PCA
y_pred_train = model.predict(x_train_pca)
accuracy_train = accuracy_score(y_true=y_train, y_pred=y_pred_train)
accuracy_train

0.7418329313746985

In [86]:
#test accuracy
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_test_pca)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
accuracy

0.5207716759837773

In [72]:
from sklearn.metrics import plot_confusion_matrix

In [73]:
plot_confusion_matrix(model, x_test, y_test)  
plt.show()  

ValueError: multilabel-indicator is not supported

In [106]:
# making our data compatible to model.
#x_train = np.expand_dims(x_train, axis=2)
#x_test = np.expand_dims(x_test, axis=2)
#x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [107]:
x_train

array([[ 1.78867448,  1.5145362 ,  1.44875584, ..., -0.17566308,
        -0.1617455 , -0.15360386],
       [-1.27257576,  0.37301025, -0.65820539, ..., -0.22554736,
        -0.21287732, -0.20400592],
       [ 2.07835725,  0.61000927,  0.2370615 , ..., -0.2122487 ,
        -0.20791637, -0.20245501],
       ...,
       [ 0.27679   ,  0.25155924,  0.44317351, ..., -0.18887466,
        -0.17612799, -0.16585193],
       [-0.51122219,  0.54016688,  0.63991329, ..., -0.22563789,
        -0.21301984, -0.20417274],
       [-0.01761639,  0.90060402,  0.54478415, ..., -0.22556401,
        -0.21299148, -0.20417192]])

In [108]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

model = RandomForestClassifier()

cv = cross_validate(model, x_train, y_train, cv = 5)

In [109]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [110]:
cv["test_score"].mean()

0.32470966349504227